In [8]:
"1번"

def solution(cards1, cards2):
    answer = 0

    for i in range(len(cards1)):
        check_dup = cards1[i].copy()
        check_dup.extend(cards2[i])
        if len(set(check_dup)) != 10:
            answer+=1
            continue
        if i != 0:
            check_dup_before_round_1 = [card for card in cards1[i] if card in cards1[i-1]]
            check_dup_before_round_2 = [card for card in cards2[i] if card in cards2[i-1]]
            if len(check_dup_before_round_1) >= 2 or len(check_dup_before_round_2) >= 2:
                answer +=1
    return answer

solution([[13, 21, 24, 29, 50], [1, 12, 20, 21, 32], [16, 26, 34, 46, 52], [9, 11, 16, 16, 21], [3, 8, 10, 16, 20]],\
   [[5, 10, 15, 41, 49], [6, 14, 15, 19, 46], [5, 42, 43, 51, 52], [5, 6, 11, 13, 45], [5, 9, 11, 13, 45]])

3

In [16]:
"2번"

from copy import deepcopy

def solution(balance, transaction, abnormal):
    answer = []
    
    for user_num in range(1, len(balance)+1):
        globals()[f'user_{user_num}'] = [{"from":user_num, "amount":balance[user_num-1]}]
        
    for tr in transaction:
        fr = tr[0]
        to = tr[1]
        am = tr[2]
        
        while am:
            cur = globals()[f'user_{fr}'].pop()
            
            if cur['amount'] > am:
                change = deepcopy(cur)
                change['amount'] -= am
                cur['amount'] = am
                globals()[f'user_{to}'].append(cur)
                globals()[f'user_{fr}'].append(change)
                am = 0
            else:
                globals()[f'user_{to}'].append(cur)
                am -= cur['amount']
                
    for num in range(1, len(balance)+1):
        globals()[f'user_{num}'] = [wallet['amount'] for wallet in globals()[f'user_{num}'] if wallet['from'] not in abnormal]
        answer.append(sum(globals()[f'user_{num}']))
        
    return answer

# solution([30, 30, 30, 30], [[1, 2, 10], [2, 3, 20], [3, 4, 5], [3, 4, 30]], [2,3])
solution([40, 30, 50], [[1, 3, 20], [2, 1, 10], [3, 1, 45], [2, 3, 10], [1, 3, 35], [2, 1, 5], [3, 1, 10], [3, 2, 5]], [2])

[40, 5, 45]

In [72]:
"3번"

from collections import deque

def rebuild_maze_handle_easy(maze, pos_to_go):
    maze_extended = [[True for _ in range(len(maze[0])+2) ] for _ in range(len(maze)+2)]
    for y in range(len(maze)):
        for x in range(len(maze[0])):
            if maze[y][x] in pos_to_go:
                maze_extended[y+1][x+1] = False
            else:
                maze_extended[y+1][x+1] = True
    return maze_extended
    
def get_able_pos(maze, cur, deq, cnt):
    pos_able_to_go = []
    if not maze[cur[0]-1][cur[1]] and (cur[0]-1, cur[1]) not in deq:
        pos_able_to_go.append([[cur[0]-1, cur[1]], cnt+1])
        
    if not maze[cur[0]][cur[1]-1] and (cur[0], cur[1]-1) not in deq:
        pos_able_to_go.append([[cur[0], cur[1]-1], cnt+1])
        
    if not maze[cur[0]+1][cur[1]] and (cur[0]+1, cur[1]) not in deq:
        pos_able_to_go.append([[cur[0]+1, cur[1]], cnt+1])
        
    if not maze[cur[0]][cur[1]+1] and (cur[0], cur[1]+1) not in deq:
        pos_able_to_go.append([[cur[0], cur[1]+1], cnt+1])      
    return pos_able_to_go
    
def get_distance_minimum_with_bfs(maze, start_point, finish_point):
    deq = deque([[start_point, 1]])
    while deq:
        cur = deq.popleft()
        distance = cur[1]
        y = cur[0][0]
        x = cur[0][1]
        if not maze[y][x]:
            maze[y][x] = distance
            deq.extend(get_able_pos(maze, (y,x), deq, distance))
        if maze[finish_point[0]][finish_point[1]]:
            return distance
    return -1 

def solution(maze, queries):
    answer = []
    init_maze = maze.copy()
    for query in queries:
        query = query.split()
        start_point = tuple(map(int, [query[:2][0], query[:2][1]]))
        finish_point = tuple(map(int, [query[2:4][0], query[2:4][1]]))
        pos_to_go = list(query[-1])
        maze = rebuild_maze_handle_easy(init_maze, pos_to_go)
        answer.append(get_distance_minimum_with_bfs(maze, start_point, finish_point))
    return answer

solution(["AAAAA", "AABBB", "CAEFG", "AAEFF"], ["1 1 3 4 AEF"])
# solution(["AAA", "ABB", "ABA"], ["1 1 1 3 A", "1 3 3 1 A", "1 1 3 3 A", "1 1 3 3 AB"])

[6]

In [70]:
"4번"

def check_benefit(remain_player, power, benefit_point):
    if len(remain_player) < benefit_point:
        return False, 1, False
    
    for i, v in enumerate(remain_player[:benefit_point]):
        if power >= v:
            power += i+1
        else :
            return False, i, False
    return power, i+1, True

def get_benefit_point(k):
    return 1 + (k-1)*2

def solution(players, power, k):
    answer = 0
    benefit_point = get_benefit_point(k)
    is_first_victory = True
    victory_point = 1
    i = 0
    while i < len(players):
        if power >= players[i] and is_first_victory:
            check_power, check_victory_point, check_victory = check_benefit(players[i:], power, benefit_point)
            if check_victory:
                power = check_power
                victory_point = check_victory_point+1
                i += check_victory_point-1
                is_first_victory = False
            else:
                power += (k * check_victory_point)
                i += check_victory_point-1
                
        elif power >= players[i] and not is_first_victory:
            power += victory_point
            victory_point+=1      
             
        else:
            power += k
            is_first_victory = True
            victory_point = 1
        i += 1
        
    answer = power    
    return answer

# solution([10, 11, 15, 14, 16, 18, 19, 20], 10, 2)
# solution([1, 2, 4, 7], 1, 2)
solution([1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2], 2, 5)


80

In [4]:
import numpy as np 
import time 

test_1 = [i for i in range(10000000)]
test_2 = np.array([i for i in range(10000000)])

s1 = time.time()
[i-10 for i in test_1]
print(time.time()- s1)
s2 = time.time()
test_2 - 10
print(time.time()- s2)

0.5261356830596924
0.01360321044921875
